# Explainable Generative AI for Stress and Burnout Risk Prediction in Healthcare Workers
## Phase 1 & 2: Data Collection, Integration, and Predictive Modeling

**Team Members:**
- Nandeesh N B - ENG23AM0047
- N Rohith - ENG23AM0046
- Niharika N - ENG23AM0048
- Nishvika Teja Reddy - ENG23AM0050

**Course:** 23AM3609 - Generative AI  
**Institution:** Dayananda Sagar University

---
## Phase 1: Data Collection and Integration

### Objectives:
1. Load and explore physiological data (Stress-Lysis.csv)
2. Load and explore workplace survey data (Workplace_Survey_Data.xlsx)
3. Integrate both datasets
4. Perform data preprocessing and feature engineering
5. Compute department-wise stress baselines

### 1.1 Import Required Libraries

In [ ]:
# Data manipulation and analysis libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning libraries - preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Machine Learning models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier

# Statistical analysis
from scipy.stats import pearsonr, spearmanr

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Set random seed for reproducibility
np.random.seed(42)

print("✓ All libraries imported successfully!")

### 1.2 Load Physiological Data (Stress-Lysis Dataset)

In [ ]:
# Load the physiological stress data
# This dataset contains: Humidity, Temperature, Step_count, and Stress_Level
physio_data = pd.read_csv('Stress-Lysis.csv')

# Display basic information about the dataset
print("="*70)
print("PHYSIOLOGICAL DATA (Stress-Lysis) - OVERVIEW")
print("="*70)
print(f"\nDataset Shape: {physio_data.shape[0]} rows × {physio_data.shape[1]} columns")
print(f"\nColumn Names: {physio_data.columns.tolist()}")
print(f"\nData Types:\n{physio_data.dtypes}")
print(f"\nMissing Values:\n{physio_data.isnull().sum()}")
print(f"\nFirst 5 Rows:")
print(physio_data.head())
print(f"\nStatistical Summary:")
print(physio_data.describe())

### 1.3 Load Workplace Survey Data

In [ ]:
# Load the workplace survey data
# This dataset contains: workhours, patients_attended, department, and dept_stress
workplace_data = pd.read_excel('Workplace_Survey_Data.xlsx')

# Display basic information about the workplace dataset
print("="*70)
print("WORKPLACE SURVEY DATA - OVERVIEW")
print("="*70)
print(f"\nDataset Shape: {workplace_data.shape[0]} rows × {workplace_data.shape[1]} columns")
print(f"\nColumn Names: {workplace_data.columns.tolist()}")
print(f"\nData Types:\n{workplace_data.dtypes}")
print(f"\nMissing Values:\n{workplace_data.isnull().sum()}")
print(f"\nDepartments: {workplace_data['department'].unique()}")
print(f"\nNumber of unique departments: {workplace_data['department'].nunique()}")
print(f"\nFirst 5 Rows:")
print(workplace_data.head())
print(f"\nStatistical Summary:")
print(workplace_data.describe())

### 1.4 Exploratory Data Analysis - Physiological Data

In [ ]:
# Visualize the distribution of stress levels in physiological data
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Stress Level Distribution
stress_counts = physio_data['Stress_Level'].value_counts().sort_index()
axes[0, 0].bar(stress_counts.index, stress_counts.values, color=['green', 'orange', 'red'])
axes[0, 0].set_xlabel('Stress Level', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Distribution of Stress Levels (0=Low, 1=Medium, 2=High)', fontsize=14, fontweight='bold')
axes[0, 0].set_xticks([0, 1, 2])
axes[0, 0].set_xticklabels(['Low', 'Medium', 'High'])
for i, v in enumerate(stress_counts.values):
    axes[0, 0].text(i, v + 10, str(v), ha='center', fontweight='bold')

# Plot 2: Temperature Distribution by Stress Level
for stress_level in sorted(physio_data['Stress_Level'].unique()):
    data_subset = physio_data[physio_data['Stress_Level'] == stress_level]['Temperature']
    axes[0, 1].hist(data_subset, alpha=0.6, label=f'Level {stress_level}', bins=20)
axes[0, 1].set_xlabel('Temperature (°F)', fontsize=12)
axes[0, 1].set_ylabel('Frequency', fontsize=12)
axes[0, 1].set_title('Temperature Distribution by Stress Level', fontsize=14, fontweight='bold')
axes[0, 1].legend()

# Plot 3: Humidity Distribution by Stress Level
for stress_level in sorted(physio_data['Stress_Level'].unique()):
    data_subset = physio_data[physio_data['Stress_Level'] == stress_level]['Humidity']
    axes[1, 0].hist(data_subset, alpha=0.6, label=f'Level {stress_level}', bins=20)
axes[1, 0].set_xlabel('Humidity (%)', fontsize=12)
axes[1, 0].set_ylabel('Frequency', fontsize=12)
axes[1, 0].set_title('Humidity Distribution by Stress Level', fontsize=14, fontweight='bold')
axes[1, 0].legend()

# Plot 4: Step Count Distribution by Stress Level
for stress_level in sorted(physio_data['Stress_Level'].unique()):
    data_subset = physio_data[physio_data['Stress_Level'] == stress_level]['Step_count']
    axes[1, 1].hist(data_subset, alpha=0.6, label=f'Level {stress_level}', bins=20)
axes[1, 1].set_xlabel('Step Count', fontsize=12)
axes[1, 1].set_ylabel('Frequency', fontsize=12)
axes[1, 1].set_title('Step Count Distribution by Stress Level', fontsize=14, fontweight='bold')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# Display percentage distribution
print("\nStress Level Distribution (Percentage):")
print((physio_data['Stress_Level'].value_counts(normalize=True) * 100).round(2))

### 1.5 Exploratory Data Analysis - Workplace Survey Data

In [ ]:
# Visualize workplace survey data
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Department Distribution
dept_counts = workplace_data['department'].value_counts()
axes[0, 0].barh(dept_counts.index, dept_counts.values, color='skyblue')
axes[0, 0].set_xlabel('Frequency', fontsize=12)
axes[0, 0].set_ylabel('Department', fontsize=12)
axes[0, 0].set_title('Distribution of Healthcare Workers by Department', fontsize=14, fontweight='bold')
for i, v in enumerate(dept_counts.values):
    axes[0, 0].text(v + 0.5, i, str(v), va='center', fontweight='bold')

# Plot 2: Work Hours Distribution
axes[0, 1].hist(workplace_data['workhours'], bins=15, color='coral', edgecolor='black')
axes[0, 1].set_xlabel('Work Hours', fontsize=12)
axes[0, 1].set_ylabel('Frequency', fontsize=12)
axes[0, 1].set_title('Distribution of Work Hours', fontsize=14, fontweight='bold')
axes[0, 1].axvline(workplace_data['workhours'].mean(), color='red', linestyle='--', 
                    label=f"Mean: {workplace_data['workhours'].mean():.1f} hrs")
axes[0, 1].legend()

# Plot 3: Patients Attended Distribution
axes[1, 0].hist(workplace_data['patients_attended'], bins=15, color='lightgreen', edgecolor='black')
axes[1, 0].set_xlabel('Patients Attended', fontsize=12)
axes[1, 0].set_ylabel('Frequency', fontsize=12)
axes[1, 0].set_title('Distribution of Patients Attended', fontsize=14, fontweight='bold')
axes[1, 0].axvline(workplace_data['patients_attended'].mean(), color='red', linestyle='--',
                   label=f"Mean: {workplace_data['patients_attended'].mean():.1f}")
axes[1, 0].legend()

# Plot 4: Department Stress Levels
dept_stress = workplace_data.groupby('department')['dept_stress'].mean().sort_values(ascending=False)
colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(dept_stress)))
axes[1, 1].barh(dept_stress.index, dept_stress.values, color=colors)
axes[1, 1].set_xlabel('Average Department Stress Level', fontsize=12)
axes[1, 1].set_ylabel('Department', fontsize=12)
axes[1, 1].set_title('Average Stress Level by Department', fontsize=14, fontweight='bold')
for i, v in enumerate(dept_stress.values):
    axes[1, 1].text(v + 0.1, i, f'{v:.1f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Print department-wise statistics
print("\nDepartment-wise Statistics:")
print("="*70)
dept_stats = workplace_data.groupby('department').agg({
    'workhours': ['mean', 'std'],
    'patients_attended': ['mean', 'std'],
    'dept_stress': ['mean', 'std']
}).round(2)
print(dept_stats)

### 1.6 Data Integration and Preprocessing

In [ ]:
# Since the datasets have different sizes, we need to create a combined dataset
# Strategy: Replicate workplace data to match the size of physiological data

# Calculate how many times we need to replicate workplace data
physio_size = len(physio_data)
workplace_size = len(workplace_data)
replication_factor = int(np.ceil(physio_size / workplace_size))

print(f"Physiological data size: {physio_size}")
print(f"Workplace data size: {workplace_size}")
print(f"Replication factor: {replication_factor}")

# Replicate and shuffle workplace data
workplace_replicated = pd.concat([workplace_data] * replication_factor, ignore_index=True)
workplace_replicated = workplace_replicated.sample(n=physio_size, random_state=42).reset_index(drop=True)

print(f"\nReplicated workplace data size: {len(workplace_replicated)}")

# Combine the datasets
combined_data = pd.concat([physio_data, workplace_replicated], axis=1)

print(f"\nCombined dataset shape: {combined_data.shape}")
print(f"\nCombined dataset columns: {combined_data.columns.tolist()}")
print(f"\nFirst 5 rows of combined data:")
print(combined_data.head())

# Check for missing values
print(f"\nMissing values in combined dataset:")
print(combined_data.isnull().sum())

### 1.7 Feature Engineering

In [ ]:
# Create new features based on domain knowledge

# 1. Temperature-Humidity Interaction
combined_data['temp_humidity_ratio'] = combined_data['Temperature'] / (combined_data['Humidity'] + 1)

# 2. Workload intensity (patients per hour)
combined_data['workload_intensity'] = combined_data['patients_attended'] / (combined_data['workhours'] + 1)

# 3. Physical activity indicator
combined_data['activity_level'] = pd.cut(combined_data['Step_count'], 
                                          bins=[0, 50, 100, 200], 
                                          labels=['Low', 'Medium', 'High'])

# 4. Work hours category
combined_data['shift_type'] = pd.cut(combined_data['workhours'],
                                      bins=[0, 8, 12, 24],
                                      labels=['Regular', 'Extended', 'Double'])

# 5. Encode categorical variables
le_dept = LabelEncoder()
combined_data['department_encoded'] = le_dept.fit_transform(combined_data['department'])

le_activity = LabelEncoder()
combined_data['activity_level_encoded'] = le_activity.fit_transform(combined_data['activity_level'])

le_shift = LabelEncoder()
combined_data['shift_type_encoded'] = le_shift.fit_transform(combined_data['shift_type'])

# Create stress level labels (Low=0, Medium=1, High=2)
combined_data['stress_category'] = combined_data['Stress_Level'].map({
    0: 'Low',
    1: 'Medium', 
    2: 'High'
})

print("Feature Engineering Completed!")
print(f"\nNew features created:")
print("- temp_humidity_ratio")
print("- workload_intensity")
print("- activity_level (categorical)")
print("- shift_type (categorical)")
print("- Encoded versions of categorical variables")

print(f"\nUpdated dataset shape: {combined_data.shape}")
print(f"\nSample of engineered features:")
print(combined_data[['Temperature', 'Humidity', 'temp_humidity_ratio', 
                     'patients_attended', 'workhours', 'workload_intensity',
                     'activity_level', 'shift_type']].head(10))

### 1.8 Correlation Analysis

In [ ]:
# Select numerical features for correlation analysis
numerical_features = ['Humidity', 'Temperature', 'Step_count', 'workhours', 
                      'patients_attended', 'dept_stress', 'temp_humidity_ratio',
                      'workload_intensity', 'department_encoded', 
                      'activity_level_encoded', 'shift_type_encoded', 'Stress_Level']

# Calculate correlation matrix
correlation_matrix = combined_data[numerical_features].corr()

# Visualize correlation heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of All Features with Stress Level', 
          fontsize=16, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Identify highly correlated features with Stress Level
stress_correlations = correlation_matrix['Stress_Level'].sort_values(ascending=False)
print("\nCorrelation with Stress Level (sorted):")
print("="*50)
print(stress_correlations)

# Identify highly correlated feature pairs (multicollinearity)
print("\n\nHighly Correlated Feature Pairs (|r| > 0.8):")
print("="*50)
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            high_corr_pairs.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                correlation_matrix.iloc[i, j]
            ))

if high_corr_pairs:
    for pair in high_corr_pairs:
        print(f"{pair[0]} <-> {pair[1]}: {pair[2]:.3f}")
else:
    print("No highly correlated pairs found (threshold: |r| > 0.8)")

### 1.9 Feature Importance using Multiple Linear Regression

In [ ]:
# Perform Multiple Linear Regression to check feature importance
# This helps understand which features linearly predict stress levels

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Prepare features (X) and target (y)
feature_cols = ['Humidity', 'Temperature', 'Step_count', 'workhours', 
                'patients_attended', 'dept_stress', 'temp_humidity_ratio',
                'workload_intensity', 'department_encoded', 
                'activity_level_encoded', 'shift_type_encoded']

X_reg = combined_data[feature_cols]
y_reg = combined_data['Stress_Level']

# Split data for regression
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# Train Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train_reg, y_train_reg)

# Make predictions
y_pred_reg = lr_model.predict(X_test_reg)

# Evaluate regression model
r2 = r2_score(y_test_reg, y_pred_reg)
rmse = np.sqrt(mean_squared_error(y_test_reg, y_pred_reg))
mae = mean_absolute_error(y_test_reg, y_pred_reg)

print("Multiple Linear Regression Results:")
print("="*50)
print(f"R² Score: {r2:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")

# Feature coefficients (importance)
feature_importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Coefficient': lr_model.coef_,
    'Abs_Coefficient': np.abs(lr_model.coef_)
}).sort_values('Abs_Coefficient', ascending=False)

print("\nFeature Coefficients (Linear Regression):")
print("="*50)
print(feature_importance_df.to_string(index=False))

# Visualize feature importance
plt.figure(figsize=(12, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Coefficient'], 
         color=['red' if x < 0 else 'green' for x in feature_importance_df['Coefficient']])
plt.xlabel('Coefficient Value', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.title('Feature Importance from Linear Regression', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
plt.tight_layout()
plt.show()

# Actual vs Predicted scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test_reg, y_pred_reg, alpha=0.5, edgecolors='k')
plt.plot([y_test_reg.min(), y_test_reg.max()], 
         [y_test_reg.min(), y_test_reg.max()], 
         'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual Stress Level', fontsize=12)
plt.ylabel('Predicted Stress Level', fontsize=12)
plt.title(f'Linear Regression: Actual vs Predicted (R² = {r2:.4f})', 
          fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 1.10 Department-wise Stress Baseline Analysis

In [ ]:
# Compute department-wise stress baselines and statistics
dept_analysis = combined_data.groupby('department').agg({
    'Stress_Level': ['mean', 'std', 'min', 'max'],
    'dept_stress': ['mean', 'std'],
    'workhours': ['mean', 'std'],
    'patients_attended': ['mean', 'std'],
    'Temperature': 'mean',
    'Step_count': 'mean'
}).round(2)

print("Department-wise Stress Baseline Analysis:")
print("="*80)
print(dept_analysis)

# Visualize department-wise stress distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Average Stress Level by Department
dept_stress_mean = combined_data.groupby('department')['Stress_Level'].mean().sort_values(ascending=False)
colors_stress = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(dept_stress_mean)))
axes[0].barh(dept_stress_mean.index, dept_stress_mean.values, color=colors_stress)
axes[0].set_xlabel('Average Stress Level', fontsize=12)
axes[0].set_ylabel('Department', fontsize=12)
axes[0].set_title('Average Stress Level by Department', fontsize=14, fontweight='bold')
for i, v in enumerate(dept_stress_mean.values):
    axes[0].text(v + 0.02, i, f'{v:.2f}', va='center', fontweight='bold')

# Plot 2: Stress Level Distribution by Department (Boxplot)
combined_data.boxplot(column='Stress_Level', by='department', ax=axes[1])
axes[1].set_xlabel('Department', fontsize=12)
axes[1].set_ylabel('Stress Level', fontsize=12)
axes[1].set_title('Stress Level Distribution by Department', fontsize=14, fontweight='bold')
plt.suptitle('')  # Remove default title

plt.tight_layout()
plt.show()

# Count of stress categories by department
dept_stress_category = pd.crosstab(combined_data['department'], 
                                   combined_data['stress_category'])
print("\nStress Category Distribution by Department:")
print("="*80)
print(dept_stress_category)

# Visualize stress category distribution by department
dept_stress_category.plot(kind='bar', stacked=False, figsize=(12, 6), 
                         color=['green', 'orange', 'red'])
plt.xlabel('Department', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.title('Stress Level Categories by Department', fontsize=14, fontweight='bold')
plt.legend(title='Stress Category')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

---
## Phase 2: Predictive Modeling

### Objectives:
1. Prepare data for machine learning models
2. Develop baseline Random Forest model
3. Develop primary XGBoost model
4. Compare model performance
5. Evaluate models with comprehensive metrics

### 2.1 Prepare Data for Machine Learning

In [ ]:
# Select features for modeling
# We'll use all relevant numerical and encoded categorical features
feature_columns = [
    'Humidity', 'Temperature', 'Step_count',
    'workhours', 'patients_attended', 'dept_stress',
    'temp_humidity_ratio', 'workload_intensity',
    'department_encoded', 'activity_level_encoded', 'shift_type_encoded'
]

# Prepare feature matrix (X) and target vector (y)
X = combined_data[feature_columns]
y = combined_data['Stress_Level']

print("Feature Matrix Shape:", X.shape)
print("Target Vector Shape:", y.shape)
print("\nFeatures used for modeling:")
for i, col in enumerate(feature_columns, 1):
    print(f"{i}. {col}")

# Split data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining set size: {X_train.shape[0]} ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Testing set size: {X_test.shape[0]} ({X_test.shape[0]/len(X)*100:.1f}%)")

# Check class distribution in train and test sets
print("\nClass distribution in training set:")
print(y_train.value_counts().sort_index())
print("\nClass distribution in testing set:")
print(y_test.value_counts().sort_index())

# Feature Scaling (important for some models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✓ Data preparation completed!")
print("✓ Features scaled using StandardScaler")

### 2.2 Model 1: Random Forest Classifier (Baseline)

In [ ]:
# Train Random Forest model as baseline
print("Training Random Forest Classifier...")
print("="*70)

# Initialize Random Forest with optimized parameters
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_model.predict(X_test)
y_pred_rf_proba = rf_model.predict_proba(X_test)

print("✓ Random Forest model trained successfully!")

# Evaluate model
rf_accuracy = accuracy_score(y_test, y_pred_rf)
rf_precision = precision_score(y_test, y_pred_rf, average='weighted')
rf_recall = recall_score(y_test, y_pred_rf, average='weighted')
rf_f1 = f1_score(y_test, y_pred_rf, average='weighted')

print("\nRandom Forest Model Performance:")
print("="*70)
print(f"Accuracy:  {rf_accuracy:.4f} ({rf_accuracy*100:.2f}%)")
print(f"Precision: {rf_precision:.4f} ({rf_precision*100:.2f}%)")
print(f"Recall:    {rf_recall:.4f} ({rf_recall*100:.2f}%)")
print(f"F1-Score:  {rf_f1:.4f} ({rf_f1*100:.2f}%)")

# Detailed classification report
print("\nDetailed Classification Report:")
print("="*70)
print(classification_report(y_test, y_pred_rf, 
                          target_names=['Low Stress', 'Medium Stress', 'High Stress']))

# Confusion Matrix
cm_rf = confusion_matrix(y_test, y_pred_rf)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Low', 'Medium', 'High'],
            yticklabels=['Low', 'Medium', 'High'])
plt.title('Random Forest - Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('Actual Stress Level', fontsize=12)
plt.xlabel('Predicted Stress Level', fontsize=12)
plt.tight_layout()
plt.show()

# Feature Importance from Random Forest
feature_importance_rf = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nFeature Importance (Random Forest):")
print("="*70)
print(feature_importance_rf.to_string(index=False))

# Plot feature importance
plt.figure(figsize=(12, 6))
plt.barh(feature_importance_rf['Feature'], feature_importance_rf['Importance'], 
         color='forestgreen')
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.title('Random Forest - Feature Importance', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 2.3 Model 2: XGBoost Classifier (Primary Model)

In [ ]:
# Train XGBoost model as primary classifier
print("Training XGBoost Classifier...")
print("="*70)

# Initialize XGBoost with optimized parameters
xgb_model = XGBClassifier(
    n_estimators=150,
    max_depth=8,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    eval_metric='mlogloss'
)

# Train the model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred_xgb = xgb_model.predict(X_test)
y_pred_xgb_proba = xgb_model.predict_proba(X_test)

print("✓ XGBoost model trained successfully!")

# Evaluate model
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
xgb_precision = precision_score(y_test, y_pred_xgb, average='weighted')
xgb_recall = recall_score(y_test, y_pred_xgb, average='weighted')
xgb_f1 = f1_score(y_test, y_pred_xgb, average='weighted')

print("\nXGBoost Model Performance:")
print("="*70)
print(f"Accuracy:  {xgb_accuracy:.4f} ({xgb_accuracy*100:.2f}%)")
print(f"Precision: {xgb_precision:.4f} ({xgb_precision*100:.2f}%)")
print(f"Recall:    {xgb_recall:.4f} ({xgb_recall*100:.2f}%)")
print(f"F1-Score:  {xgb_f1:.4f} ({xgb_f1*100:.2f}%)")

# Detailed classification report
print("\nDetailed Classification Report:")
print("="*70)
print(classification_report(y_test, y_pred_xgb,
                          target_names=['Low Stress', 'Medium Stress', 'High Stress']))

# Confusion Matrix
cm_xgb = confusion_matrix(y_test, y_pred_xgb)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_xgb, annot=True, fmt='d', cmap='Oranges',
            xticklabels=['Low', 'Medium', 'High'],
            yticklabels=['Low', 'Medium', 'High'])
plt.title('XGBoost - Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('Actual Stress Level', fontsize=12)
plt.xlabel('Predicted Stress Level', fontsize=12)
plt.tight_layout()
plt.show()

# Feature Importance from XGBoost
feature_importance_xgb = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nFeature Importance (XGBoost):")
print("="*70)
print(feature_importance_xgb.to_string(index=False))

# Plot feature importance
plt.figure(figsize=(12, 6))
plt.barh(feature_importance_xgb['Feature'], feature_importance_xgb['Importance'],
         color='darkorange')
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.title('XGBoost - Feature Importance', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 2.4 Model Comparison and Analysis

In [ ]:
# Compare model performance
comparison_df = pd.DataFrame({
    'Model': ['Random Forest', 'XGBoost'],
    'Accuracy': [rf_accuracy, xgb_accuracy],
    'Precision': [rf_precision, xgb_precision],
    'Recall': [rf_recall, xgb_recall],
    'F1-Score': [rf_f1, xgb_f1]
})

print("Model Performance Comparison:")
print("="*70)
print(comparison_df.to_string(index=False))

# Calculate improvement
improvement = ((xgb_accuracy - rf_accuracy) / rf_accuracy) * 100
print(f"\nXGBoost Performance Improvement over Random Forest: {improvement:.2f}%")

# Visualize model comparison
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
rf_scores = [rf_accuracy, rf_precision, rf_recall, rf_f1]
xgb_scores = [xgb_accuracy, xgb_precision, xgb_recall, xgb_f1]

x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar(x - width/2, rf_scores, width, label='Random Forest', color='forestgreen', alpha=0.8)
bars2 = ax.bar(x + width/2, xgb_scores, width, label='XGBoost', color='darkorange', alpha=0.8)

ax.set_xlabel('Metrics', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.set_ylim([0.80, 1.0])
ax.grid(axis='y', alpha=0.3)

# Add value labels on bars
def autolabel(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}',
                   xy=(bar.get_x() + bar.get_width() / 2, height),
                   xytext=(0, 3),
                   textcoords="offset points",
                   ha='center', va='bottom',
                   fontweight='bold')

autolabel(bars1)
autolabel(bars2)

plt.tight_layout()
plt.show()

# Per-class F1-Score comparison
rf_f1_per_class = f1_score(y_test, y_pred_rf, average=None)
xgb_f1_per_class = f1_score(y_test, y_pred_xgb, average=None)

print("\nPer-Class F1-Score Comparison:")
print("="*70)
class_comparison = pd.DataFrame({
    'Stress Level': ['Low (0)', 'Medium (1)', 'High (2)'],
    'Random Forest F1': rf_f1_per_class,
    'XGBoost F1': xgb_f1_per_class
})
print(class_comparison.to_string(index=False))

# Visualize per-class F1-scores
fig, ax = plt.subplots(figsize=(10, 6))
x_pos = np.arange(3)
width = 0.35

bars1 = ax.bar(x_pos - width/2, rf_f1_per_class, width, 
               label='Random Forest', color='forestgreen', alpha=0.8)
bars2 = ax.bar(x_pos + width/2, xgb_f1_per_class, width,
               label='XGBoost', color='darkorange', alpha=0.8)

ax.set_xlabel('Stress Level', fontsize=12)
ax.set_ylabel('F1-Score', fontsize=12)
ax.set_title('Per-Class F1-Score Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(['Low', 'Medium', 'High'])
ax.legend()
ax.set_ylim([0.75, 1.0])
ax.grid(axis='y', alpha=0.3)

autolabel(bars1)
autolabel(bars2)

plt.tight_layout()
plt.show()

### 2.5 Cross-Validation Analysis

In [ ]:
# Perform 5-fold cross-validation for both models
from sklearn.model_selection import cross_validate

print("Performing 5-Fold Cross-Validation...")
print("="*70)

# Define scoring metrics
scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']

# Random Forest Cross-Validation
print("\nRandom Forest Cross-Validation:")
rf_cv_results = cross_validate(rf_model, X, y, cv=5, scoring=scoring, n_jobs=-1)

print(f"Accuracy:  {rf_cv_results['test_accuracy'].mean():.4f} (+/- {rf_cv_results['test_accuracy'].std():.4f})")
print(f"Precision: {rf_cv_results['test_precision_weighted'].mean():.4f} (+/- {rf_cv_results['test_precision_weighted'].std():.4f})")
print(f"Recall:    {rf_cv_results['test_recall_weighted'].mean():.4f} (+/- {rf_cv_results['test_recall_weighted'].std():.4f})")
print(f"F1-Score:  {rf_cv_results['test_f1_weighted'].mean():.4f} (+/- {rf_cv_results['test_f1_weighted'].std():.4f})")

# XGBoost Cross-Validation
print("\nXGBoost Cross-Validation:")
xgb_cv_results = cross_validate(xgb_model, X, y, cv=5, scoring=scoring, n_jobs=-1)

print(f"Accuracy:  {xgb_cv_results['test_accuracy'].mean():.4f} (+/- {xgb_cv_results['test_accuracy'].std():.4f})")
print(f"Precision: {xgb_cv_results['test_precision_weighted'].mean():.4f} (+/- {xgb_cv_results['test_precision_weighted'].std():.4f})")
print(f"Recall:    {xgb_cv_results['test_recall_weighted'].mean():.4f} (+/- {xgb_cv_results['test_recall_weighted'].std():.4f})")
print(f"F1-Score:  {xgb_cv_results['test_f1_weighted'].mean():.4f} (+/- {xgb_cv_results['test_f1_weighted'].std():.4f})")

# Visualize cross-validation results
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Random Forest CV scores
rf_cv_data = [rf_cv_results['test_accuracy'], 
              rf_cv_results['test_precision_weighted'],
              rf_cv_results['test_recall_weighted'],
              rf_cv_results['test_f1_weighted']]

bp1 = axes[0].boxplot(rf_cv_data, labels=['Accuracy', 'Precision', 'Recall', 'F1-Score'],
                      patch_artist=True)
for patch in bp1['boxes']:
    patch.set_facecolor('forestgreen')
    patch.set_alpha(0.6)
axes[0].set_ylabel('Score', fontsize=12)
axes[0].set_title('Random Forest - Cross-Validation Scores', fontsize=14, fontweight='bold')
axes[0].set_ylim([0.80, 1.0])
axes[0].grid(axis='y', alpha=0.3)

# XGBoost CV scores
xgb_cv_data = [xgb_cv_results['test_accuracy'],
               xgb_cv_results['test_precision_weighted'],
               xgb_cv_results['test_recall_weighted'],
               xgb_cv_results['test_f1_weighted']]

bp2 = axes[1].boxplot(xgb_cv_data, labels=['Accuracy', 'Precision', 'Recall', 'F1-Score'],
                      patch_artist=True)
for patch in bp2['boxes']:
    patch.set_facecolor('darkorange')
    patch.set_alpha(0.6)
axes[1].set_ylabel('Score', fontsize=12)
axes[1].set_title('XGBoost - Cross-Validation Scores', fontsize=14, fontweight='bold')
axes[1].set_ylim([0.80, 1.0])
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 2.6 Final Summary and Key Findings

In [ ]:
print("\n" + "="*80)
print("PHASE 1 & 2 SUMMARY - KEY FINDINGS")
print("="*80)

print("\n1. DATA COLLECTION & INTEGRATION")
print("-" * 80)
print(f"   • Physiological Data: {len(physio_data)} samples")
print(f"   • Workplace Survey Data: {len(workplace_data)} samples")
print(f"   • Combined Dataset: {len(combined_data)} samples with {len(feature_columns)} features")
print(f"   • Departments Analyzed: {', '.join(combined_data['department'].unique())}")

print("\n2. KEY CORRELATIONS WITH STRESS LEVEL")
print("-" * 80)
top_correlations = stress_correlations.head(6)
for feature, corr in top_correlations.items():
    if feature != 'Stress_Level':
        print(f"   • {feature}: {corr:.4f}")

print("\n3. DEPARTMENT-WISE STRESS ANALYSIS")
print("-" * 80)
dept_avg_stress = combined_data.groupby('department')['Stress_Level'].mean().sort_values(ascending=False)
for dept, stress in dept_avg_stress.items():
    print(f"   • {dept}: {stress:.2f} (Average Stress Level)")

print("\n4. MODEL PERFORMANCE COMPARISON")
print("-" * 80)
print(f"   Random Forest:")
print(f"     - Accuracy:  {rf_accuracy:.4f} ({rf_accuracy*100:.2f}%)")
print(f"     - Precision: {rf_precision:.4f}")
print(f"     - Recall:    {rf_recall:.4f}")
print(f"     - F1-Score:  {rf_f1:.4f}")
print(f"\n   XGBoost (Primary Model):")
print(f"     - Accuracy:  {xgb_accuracy:.4f} ({xgb_accuracy*100:.2f}%)")
print(f"     - Precision: {xgb_precision:.4f}")
print(f"     - Recall:    {xgb_recall:.4f}")
print(f"     - F1-Score:  {xgb_f1:.4f}")
print(f"\n   ✓ XGBoost outperforms Random Forest by {improvement:.2f}%")

print("\n5. TOP 5 MOST IMPORTANT FEATURES (XGBoost)")
print("-" * 80)
for i, row in feature_importance_xgb.head(5).iterrows():
    print(f"   {i+1}. {row['Feature']}: {row['Importance']:.4f}")

print("\n6. PER-CLASS PERFORMANCE (XGBoost)")
print("-" * 80)
for i, (level, f1) in enumerate(zip(['Low Stress', 'Medium Stress', 'High Stress'], xgb_f1_per_class)):
    print(f"   • {level}: F1-Score = {f1:.4f} ({f1*100:.2f}%)")

print("\n7. OBJECTIVES ACHIEVED")
print("-" * 80)
print(f"   ✓ Prediction Accuracy: {xgb_accuracy:.2%} (Target: ≥85%)")
print(f"   ✓ XGBoost vs RF Improvement: {improvement:.2f}% (Target: 5-10%)")
print(f"   ✓ F1-Score for Medium Stress: {xgb_f1_per_class[1]:.4f} (Target: >0.80)")
print(f"   ✓ F1-Score for High Stress: {xgb_f1_per_class[2]:.4f} (Target: >0.80)")
print(f"   ✓ Feature importance analysis completed")
print(f"   ✓ Department-wise stress profiling completed")

print("\n" + "="*80)
print("Phase 1 & 2 completed successfully! Ready for Phase 3 (XAI) and Phase 4 (GenAI)")
print("="*80)

---
## Next Steps

### Phase 3: Explainable AI (XAI)
- Implement SHAP (SHapley Additive exPlanations) for model interpretability
- Generate global and local feature importance explanations
- Create visualizations for model decision-making process

### Phase 4: Generative AI Integration
- Combine predictions with SHAP values
- Engineer prompts for LLMs (GPT/LLaMA/Gemini)
- Generate personalized stress reports and intervention recommendations
- Create natural language explanations of stress predictions

---
**End of Phase 1 & 2 Notebook**